In [1]:
#!/usr/bin/env python
import os
import math
import rospy
import numpy as np
from threading import Lock

from std_msgs.msg import String
from sensor_msgs.msg import LaserScan
from geometry_msgs.msg import Point
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry

import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

import rospy
import rosservice
from std_msgs.msg import String
from sensor_msgs.msg import LaserScan
from pcimr_simulation.srv import InitPos
from geometry_msgs.msg import Twist
import random
from std_srvs.srv import Empty
import numpy as np

from gazebo_msgs.srv import GetModelState, GetModelStateRequest, SetModelState, SetModelStateRequest

SPEED_X = 0.5
SPEED_Y = 0.5
ANGULAR_Z = 0.3

BATCH_SIZE = 64
GAMMA = 0.95
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10
LEARNING_RATE=0.0005


class ControllerNode:

    def __init__(self):
        # Initialize member variables
        self.sim_lock = Lock()

        self.sub_twist = rospy.Subscriber('/input/cmd_vel', Twist, self.get_and_filter_twist)
        self.sub_scan = rospy.Subscriber('/scan', LaserScan, self.get_scan)

        self.pub_twist = None
            
        self.pub_twist = rospy.Publisher('/cmd_vel', Twist, queue_size=10)
        
        self.goal_dist = 100
        self.scan = LaserScan()
        self.robot_twist = Twist()
        self.cmd_vel = Twist()
        
        #the service that lets you SET the state of a model within the gazebo environment (including position)
        rospy.wait_for_service ('/gazebo/set_model_state')
        self.set_model_srv = rospy.ServiceProxy('/gazebo/set_model_state', SetModelState)
        
        #the service that lets you GET the state of a model within the gazebo environment (including position)
        rospy.wait_for_service ('/gazebo/get_model_state')
        self.get_model_srv = rospy.ServiceProxy('/gazebo/get_model_state', GetModelState)

        #state requests withg the names of the respective model
        self.robot_model = GetModelStateRequest()
        self.robot_model.model_name='rto-1'

        self.goal_model = GetModelStateRequest()
        self.goal_model.model_name='cp'
        
        self.checkpoint_names = ['cp_marker', 'cp_marker_0', 'cp_marker_1', 'cp_marker_2', 'cp_marker_3', 'cp_marker_4', 'cp_marker_5', 'cp_marker_6']
        self.cp_index = 0



    def get_and_filter_twist(self, msg):
        self.robot_twist = msg
    
    def get_scan(self, msg):
        self.scan = msg
        
    def get_env(self):
        rospy.wait_for_message("scan", LaserScan)
        r = self.scan.ranges
        vision_lines = [r[0], r[41], r[82], r[123], r[164], r[205], r[244]]
        return torch.FloatTensor(vision_lines)
    
    #get the distance between the robot "rto-1" and "goal" object
    def check_goal_distance(self):
        result = self.get_model_srv(self.robot_model)
        robot_pos = result.pose.position

        result = self.get_model_srv(self.goal_model)
        goal_pos = result.pose.position

        diff_x = robot_pos.x - goal_pos.x
        diff_y = robot_pos.y - goal_pos.y

        return math.sqrt(diff_x*diff_x + diff_y*diff_y)
    
    def next_checkpoint(self):
        self.cp_index = (self.cp_index + 1)%len(self.checkpoint_names)
        
        get_request = GetModelStateRequest()
        get_request.model_name = self.checkpoint_names[self.cp_index]
        get_result = self.get_model_srv(get_request)

        set_request = SetModelStateRequest()
        set_request.model_state.model_name = "cp"
        set_request.model_state.pose = get_result.pose

        set_result = self.set_model_srv(set_request)
    
    '''
    def get_reward(self):
        rospy.wait_for_message("scan", LaserScan)
        if min(self.scan.ranges) < 0.20:
            self.reset_simulation = rospy.ServiceProxy('/gazebo/reset_simulation', Empty)
            self.reset_simulation()
            return -1.0
        elif min(self.scan.ranges) > 1.0:
            return -0.01 + 0.02 / self.check_goal_distance()
        elif self.check_goal_distance() < 0.2:
            self.reset_simulation = rospy.ServiceProxy('/gazebo/reset_simulation', Empty)
            self.reset_simulation()
            return +1.0
        else:
            return -0.05 +  0.02 / self.check_goal_distance()
    '''
    def get_reward(self):
        rospy.wait_for_message("scan", LaserScan)
        r = self.scan.ranges
        vision_lines = [r[0], r[41], r[82], r[123], r[164], r[205], r[244]]
        if min(vision_lines) < 0.20:
            self.reset_simulation = rospy.ServiceProxy('/gazebo/reset_simulation', Empty)
            self.reset_simulation()
            self.cp_index = 0
            return -1.0
        
        elif self.check_goal_distance() < 0.8:
            print("checkpoint reached")
            self.next_checkpoint()
            return +1.0
        
        return 0.3
        '''
        print("Speed: " + str(self.cmd_vel.linear.x))
        log_speed = math.log(self.cmd_vel.linear.x)
        
        if log_speed == 0.0:
            return -0.04
        
        else:
            return log_speed
        '''
        
    def use_action(self, nr):
        if nr == 0:
            self.cmd_vel.linear.x = SPEED_X
            self.cmd_vel.linear.y = 0.0
            self.cmd_vel.angular.z = 0.0
        if nr == 1:
            self.cmd_vel.linear.x = SPEED_X
            self.cmd_vel.linear.y = 0.0
            self.cmd_vel.angular.z = ANGULAR_Z
        if nr == 2:
            self.cmd_vel.linear.x = SPEED_X
            self.cmd_vel.linear.y = 0.0
            self.cmd_vel.angular.z = -ANGULAR_Z
        self.pub_twist.publish(self.cmd_vel)
            



In [2]:
#imports


In [3]:



# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [5]:
class DQN(nn.Module):

    def __init__(self, outputs):
        super(DQN, self).__init__()
        # TODO: Change this numbers for input
        self.conv1 = nn.Conv1d(1, 1, kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm1d(1)

        # TODO : Number of Linear input connections depends on output of conv2d layers
        # and therefore the input image size, so compute it.
#         def conv2d_size_out(size, kernel_size = 5, stride = 2):
#             return (size - (kernel_size - 1) - 1) // stride  + 1
#         convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w)))
#         convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h)))
        # TODO : Check 32 below
#         linear_input_size = convw * convh * 40
        self.fc1 = nn.Linear(7, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200, 200)
        self.fc4 = nn.Linear(200, 3)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = x.cuda()
        #x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        
        #x = x.to(self.d.device)
        
        return x

In [6]:
policy_checkpoint_path = None
target_checkpoint_path = None

# Get screen size so that we can initialize layers correctly based on shape
# returned from AI gym. Typical dimensions at this point are close to 3x40x90
# which is the result of a clamped and down-scaled render buffer in get_screen()
# init_screen = get_screen()
# Get number of actions from gym action space
n_actions = 3

# TODO: Define the parameters here correctly for our environment
policy_net = DQN(n_actions).to(device)
target_net = DQN(n_actions).to(device)

optimizer = optim.RMSprop(policy_net.parameters(), lr=LEARNING_RATE)

if policy_checkpoint_path != None:
    print("loading network")
    #load policy net
    policy_checkpoint = torch.load(policy_checkpoint_path)
    policy_net.load_state_dict(policy_checkpoint['model_state_dict'])
    optimizer.load_state_dict(policy_checkpoint['optimizer_state_dict'])
    
    #load target net
    target_checkpoint = torch.load(target_checkpoint_path)
    target_net.load_state_dict(target_checkpoint['model_state_dict'])
else:
    target_net.load_state_dict(policy_net.state_dict())

target_net.eval()

memory = ReplayMemory(10000)

steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            t = policy_net(state.to(device))
            return torch.tensor([[t.max(0)[1]]], device=device, dtype=torch.long)
    else:
        t = torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)
        return t


episode_durations = []


In [7]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])

    
    
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    state_batch=torch.reshape(state_batch, (BATCH_SIZE,7))
    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    non_final_next_states=torch.reshape(non_final_next_states, (BATCH_SIZE,7))
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))
    loss_float = loss.item()
    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.data.clamp_(-1.0, 1.0)
    optimizer.step()
    
    return loss_float

In [8]:
num_episodes = 501
reset_simulation = rospy.ServiceProxy('/gazebo/reset_simulation', Empty)
reset_simulation()
rospy.init_node('controller_node')

checkpoint_root = "/home/tobi/checkpoints/cp_tryout"

simple_sim_node = ControllerNode()

losses = []
for i_episode in range(num_episodes):
    # TODO:  Initialize the environment and state
    #env.reset()
    
    if i_episode%100 == 0 and i_episode != 0:
        print("saving model after episode " + str(i_episode))
        
        #save policy net
        torch.save({
            'model_state_dict': policy_net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
            }, checkpoint_root + "/policy_net_" + str(i_episode) + ".pth")
        
        #save target net
        torch.save({
            'model_state_dict': policy_net.state_dict()
            }, checkpoint_root + "/target_net_" + str(i_episode) + ".pth")
    
    # TODO: Implement these functions
    last_screen = simple_sim_node.get_env()
    current_screen = simple_sim_node.get_env()
    state =current_screen - last_screen
    print("starting episode " + str(i_episode))
    for t in count():
        #print(t)
        # Select and perform an action
        action = select_action(state)
        #print(f"Selected action: {action}")
        # TODO: Get values from action
        simple_sim_node.use_action(action.item())
        reward = simple_sim_node.get_reward()
        #print("Reward: " + str(reward))
        
        reward = torch.tensor([reward], device=device, dtype=torch.float)
        
        # TODO : Observe new state
        last_screen = current_screen
        current_screen = simple_sim_node.get_env()
        # If we are not done get next state
        next_state = current_screen - last_screen

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state
        #print(f"Optimizing model:")
        # Perform one step of the optimization (on the target network)
        loss = optimize_model()
        losses.append(loss)
        if reward == -1.0 or reward == +1.0:
            episode_durations.append(t + 1)
            # TODO : Plotting 
            # plot_durations()
            break
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')
plt.plot(losses)
# env.render()
# env.close()
# plt.ioff()
# plt.show()

starting episode 0
starting episode 1


ROSInterruptException: rospy shutdown